In [1]:
import csv
import os
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import numpy as np
import pyarrow

In [2]:
os.getcwd()

'/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3'

In [3]:
df = dd.read_csv('/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/data/spam_test/SMSSpamCollection.tsv',
                delimiter='\t', dtype='object')

In [4]:
df.head()

,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [5]:
new_columns = ['emailtype', 'text']
df = df.rename(columns=dict(zip(df.columns, new_columns)))

In [6]:
with ProgressBar():
    df.repartition(npartitions=1).to_csv('/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/part2*.csv',
                                        index=False)

[########################################] | 100% Completed |  0.1s


In [7]:
df_cleaned = dd.read_csv('/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/part2*.csv')

In [8]:
df_cleaned.head()

,emailtype,text
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [11]:
#creating a condition whereby we specify 1 if spam or 2 if not
type_parsed = df_cleaned['emailtype'].apply(lambda x: "1" if x=='spam' else "2", meta=('int64'))
df_derived_stage1 = df_cleaned.drop('emailtype', axis=1)
df_derived_stage2 = df_derived_stage1.assign(Emailtype=type_parsed)
#df_derived_stage3 = df_derived_stage2.rename(columns={'Emailtype':'Email Type'})

In [12]:
text = df_cleaned['text'].apply(lambda x: x, meta='object')
df_derived_stage3 = df_derived_stage2.drop('text', axis=1) #drop the original x axis 
df_derived_stage4 = df_derived_stage3.assign(TextValues=text)

In [13]:
#name the columns back
df_derived_stage5 = df_derived_stage4.rename(columns={'Emailtype': 'Email Type', 'TextValues': 'Text'})

In [14]:
#cleaned_df = dd.read_csv('part*.csv')
df_derived_stage5.to_csv('part0*.csv', index=False)

['/Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/part00.csv']

In [16]:
df_derived_stage5.head()

,Email Type,Text
0,2,Ok lar... Joking wif u oni...
1,1,Free entry in 2 a wkly comp to win FA Cup fina...
2,2,U dun say so early hor... U c already then say...
3,2,"Nah I don't think he goes to usf, he lives aro..."
4,1,FreeMsg Hey there darling it's been 3 week's n...


In [18]:
#split into two csvs
import csv
lines = []
with open('part00.csv') as in_file:
    ts = csv.reader(in_file)
    for i in ts:
        lines.append(i)

In [36]:
#with ProgressBar():
#    email_types = cleaned_df['emailtype'].value_counts()
#email_types.compute()
lines[2]

['1',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"]

In [21]:
lines_train = int(len(lines) * 0.9)

In [22]:
lines_train

5014

In [37]:
with open('//Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/spam_dataset/spam_train.csv', 'w') as out_file:
    ts2 = csv.writer(out_file)
    ts2.writerows(lines[:lines_train])

In [ ]:
with open('//Users/thomassullivan/projects/GitHub/PyTorchNLPBook/chapters/chapter_3/spam_dataset/spam_test.csv', 'w') as out_file:
    ts2 = csv.writer(out_file)
    ts2.writerows(lines[lines_train:])
